# Imports

In [ ]:
import numpy as np
import pandas as pd
from asapdiscovery.docking import analysis as a
from importlib import reload
reload(a)
from asapdiscovery.data.schema_v2.ligand import Ligand
from asapdiscovery.data.schema_v2.complex import Complex
from asapdiscovery.data.selectors.mcs_selector import MCSSelector
from openeye import oegraphsim
from tqdm import tqdm
from pathlib import Path
import json
from itertools import combinations # only need to do the combinations once since the Tanimoto is symmetric

## Load the data

In [ ]:
data_path = Path("../../data/20240202_fragalysis_p_series_schema")

In [ ]:
complexes = [Complex.from_dict(json.load(open(p))) for p in data_path.glob("*.json")]

In [ ]:
mols = [c.ligand for c in complexes]

In [ ]:
def get_fp(mol, bit_size=2048, radius=2):
    fp = oegraphsim.OEFingerPrint()
    oegraphsim.OEMakeCircularFP(fp, mol, bit_size, 0, radius, oegraphsim.OEFPAtomType_DefaultCircularAtom, oegraphsim.OEFPBondType_DefaultCircularBond)
    return fp

In [ ]:
oegraphsim.OEGetFPAtomType(oegraphsim.OEFPAtomType_DefaultCircularAtom)

In [ ]:
fps = [get_fp(mol.to_oemol()) for mol in mols]

In [ ]:
def calculate_tanimoto(fp1, fp2):
    return oegraphsim.OETanimoto(fp1, fp2)

In [ ]:
tc_list = [calculate_tanimoto(fp1, fp2) for fp1 in fps for fp2 in fps]

In [ ]:
tc_matrix = np.array(tc_list).reshape(len(fps), len(fps))

In [ ]:
import plotly.express as px

In [ ]:
px.imshow(tc_matrix)

In [ ]:
df = pd.DataFrame({"Mol1": [m.compound_name for m in mols for _ in mols], "Mol2": [m.compound_name for _ in mols for m in mols], "Tanimoto": tc_list})

In [ ]:
df_filtered = df[df["Mol1"] != df["Mol2"]]

In [ ]:
top10_df = df_filtered.groupby(['Mol1']).head(10)

In [ ]:
top_df = df_filtered.groupby(['Mol1']).head(1)

In [ ]:
mean_df = top10_df.groupby(['Mol1'])[["Tanimoto"]].mean().reset_index()

In [ ]:
px.strip(top10_df, x="Mol1", y="Tanimoto", category_orders={"Mol1": mean_df.sort_values("Tanimoto")["Mol1"]})

In [ ]:
def get_tc_df(mols, bit_size=2048, radius=2):
    fps = [get_fp(mol.to_oemol(), bit_size, radius) for mol in mols]
    tc_list = [calculate_tanimoto(fp1, fp2) for fp1 in fps for fp2 in fps]
    df = pd.DataFrame({"Mol1": [m.compound_name for m in mols for _ in mols], 
                       "Mol2": [m.compound_name for _ in mols for m in mols], 
                       "Tanimoto": tc_list,
                       "Fingerprint": f"ECFP{2*radius}",
                       "BitSize": f"{bit_size} bits"})
    df_filtered = df[df["Mol1"] != df["Mol2"]]
    return df_filtered

In [ ]:
dfs = []
for radius in [2, 3, 4, 5]:
    for bit_size in [2048]:
        df = get_tc_df(mols, bit_size, radius)
        dfs.append(df)

In [ ]:
all_df = pd.concat(dfs)

In [ ]:
all_df.to_csv("tanimoto_fingerprint_comparison.csv", index=False)

In [ ]:
all_top_df = all_df.groupby(['Mol1', 'Fingerprint']).head(1)

In [ ]:
px.histogram(all_top_df, 
             x="Tanimoto", 
             nbins=20, 
             title="Tanimoto similarity between the most similar ligands", 
             range_x=[0, 1], 
             template="plotly_white", 
             labels={"Tanimoto": "Tanimoto similarity (ECFP4 Fingerprints)"},
                color="Fingerprint",
             barmode="overlay",
             opacity=0.75)

## kde distplot

In [ ]:
import plotly.figure_factory as ff

In [ ]:
hist_data = [all_top_df[all_top_df["Fingerprint"] == fp]["Tanimoto"] for fp in all_top_df.Fingerprint.unique()]

In [ ]:
fig = ff.create_distplot(hist_data, 
                         group_labels=[fp for fp in all_top_df.Fingerprint.unique()], 
                         bin_size=0.1, 
                         histnorm="probability", 
                         show_rug=False,
                         show_hist=False)

In [ ]:
fig.update_xaxes(range=[0, 1])
fig.update_yaxes(range=[0, 1])
fig.update_layout(template="simple_white", 
                  title="Maximum Tanimoto similarities in this dataset", 
                  xaxis_title="Tanimoto similarity", 
                  yaxis_title="Probability", 
                  height=400, 
                  width=600,
                  legend=dict(title="Fingerprint Type", y=0.5, x=0.8))
fig.write_image("tanimoto_kde.svg")

## add MCSS to the mix

In [ ]:
mcss_hist_data = np.load("mcss_tanimoto.npy")

In [ ]:
fig = ff.create_distplot(hist_data + [mcss_hist_data], 
                         group_labels=[fp for fp in all_top_df.Fingerprint.unique()] + ["Number of Atoms in the MCSS"], 
                         bin_size=0.1, 
                         histnorm="probability", 
                         show_rug=False,
                         show_hist=False)

In [ ]:
fig.update_xaxes(range=[0, 1])
fig.update_yaxes(range=[0, 1])
fig.update_layout(template="simple_white", 
                  title="Maximum Tanimoto similarities in this dataset", 
                  xaxis_title="Tanimoto similarity", 
                  yaxis_title="Probability", 
                  height=400, 
                  width=600,
                  legend=dict(title="Fingerprint Type", y=0.5, x=0.8))
fig.write_image("tanimoto_kde.svg")

In [ ]:
mol1 = mols[72]
mol2 = mols[52]

In [ ]:
from rdkit import Chem

In [ ]:
Chem.MolFromSmiles(mol1.smiles)

In [ ]:
Chem.MolFromSmiles(mol2.smiles)

In [ ]:
from rdkit.Chem import MACCSkeys, rdFingerprintGenerator, DataStructs

In [ ]:
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

In [ ]:
fp1 = fpg.GetFingerprint(Chem.MolFromSmiles(mol1.smiles))

In [ ]:
fp2 = fpg.GetFingerprint(Chem.MolFromSmiles(mol2.smiles))

In [ ]:
DataStructs.TanimotoSimilarity(fp1, fp2)

In [ ]:
tc_matrix[97, 143]

In [ ]:
tc_matrix[143, 97]

In [ ]:
fptype = oegraphsim.OEGetFPType(oegraphsim.OEFPType_Circular)
print(fptype.GetFPTypeString())